In [1]:
import sys
sys.path.append( "/home/peer/PycharmProjects/neslearng")

In [2]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper
from hsa.visualization.parse_fm2 import parse_fm2
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import importlib
importlib.reload(rewards)

<module 'hsa.ba.rewards' from '/home/peer/PycharmProjects/neslearng/hsa/ba/rewards.py'>

In [3]:
%matplotlib 

Using matplotlib backend: TkAgg


In [4]:
movie_paths = !ls ../../movies/*.*
movie_names = !ls ../../movies/

In [5]:
# just checking if the match up
list(zip(movie_names, movie_paths))

[('10_1-1_slow_with_powerup.fm2', '../../movies/10_1-1_slow_with_powerup.fm2'),
 ('11_1-1_small_walking.fm2', '../../movies/11_1-1_small_walking.fm2'),
 ('12_1-1_dieing_to_first_gumba.fm2',
  '../../movies/12_1-1_dieing_to_first_gumba.fm2'),
 ('13_1-1_dieing_to_gumbas.fm2', '../../movies/13_1-1_dieing_to_gumbas.fm2'),
 ('5_1-1_without-shortcut.fm2', '../../movies/5_1-1_without-shortcut.fm2'),
 ('6_1-1_with_shortcut.fm2', '../../movies/6_1-1_with_shortcut.fm2'),
 ('7_1-1_death_and_checkpoint_no_shortcut.fm2',
  '../../movies/7_1-1_death_and_checkpoint_no_shortcut.fm2'),
 ('8_1-1_death_and_shortcut.fm2', '../../movies/8_1-1_death_and_shortcut.fm2'),
 ('9_1-1_death_in_shortcut.fm2', '../../movies/9_1-1_death_in_shortcut.fm2')]

In [18]:
reward_functions = {
    "fine_potential":rewards.make_finer_potential,
    "delta_points":rewards.make_delta_points,
    "time_left":lambda:rewards.scaled_for_time_left,
    "finer_with_points": rewards.make_finer_main_reward_with_points
}

In [23]:
def reward_from_movie(movie_path):
    reward_func_instances = {name: factory() for name, factory in reward_functions.items()}
    nes = NESInterface(hsa.machine_constants.mario_rom_location, eb_compatible=False, auto_render_period=3, reward_type="ehrenbrav")
    with open(movie_path) as movie_file:
        inputs_from_movie = list(parse_fm2(movie_file))
    movie_length = len(inputs_from_movie)
    
    df = pandas.DataFrame(index=range(movie_length),
                          columns=(list(reward_functions.keys())+["ehrenbrav"]),
                          dtype = np.float)
    
    for i, combi in enumerate(inputs_from_movie):
        reward = nes.act(py_to_nes_wrapper(combi))
        if i < 6:
            continue # ignore the first few frames on the start menu, because the rewards are not valid there
        df.at[i,"ehrenbrav"] = reward/100
        ram = nes.getRAM()
        for name, func in reward_func_instances.items():
            df.at[i,name] = func(ram)        
    del nes
    return df


In [24]:
reward_frames = {}
for movie_name, movie_path in zip(movie_names,movie_paths):
    reward_frames[movie_name] = reward_from_movie(movie_path)

In [32]:
# scale down ehrenbrav
for movie_name, frame in reward_frames.items():
    frame["ehrenbrav"] = frame["ehrenbrav"].map(lambda x: x/5)

In [20]:
for movie_name, frame in reward_frames.items():
    movie_name_no_ending = movie_name[:-4]
    frame.plot(subplots=True,figsize=(10, 20))
    plt.savefig("ehrenbrav-pics/{}-all.png".format(movie_name_no_ending),format="png")
    plt.clf()

In [33]:
reward_frames["5_1-1_without-shortcut.fm2"].plot()

In [34]:
reward_frames["8_1-1_death_and_shortcut.fm2"].plot()

### More reward evalutation

In [ ]:
reward_frames.keys()

In [ ]:
reward_frames["5_1-1_without-shortcut.fm2"]["time_left_coarse_fine_potential"].plot()